In [11]:
import pandas as pd
import h2o
from h2o.estimators import H2ORandomForestEstimator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score

In [1]:
# download the dataset from Kaggle
!kaggle datasets download -d uciml/adult-census-income -p ../Data

Dataset URL: https://www.kaggle.com/datasets/uciml/adult-census-income
License(s): CC0-1.0
  0% 0.00/450k [00:00<?, ?B/s]
100% 450k/450k [00:00<00:00, 21.7MB/s]


In [2]:
# unzip the dataset
!unzip ../Data/adult-census-income.zip -d ../Data

Archive:  ../Data/adult-census-income.zip
  inflating: ../Data/adult.csv       


In [26]:
income = pd.read_csv("../Data/adult.csv")
income

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,90,?,77053,HS-grad,9,Widowed,?,Not-in-family,White,Female,0,4356,40,United-States,<=50K
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K
2,66,?,186061,Some-college,10,Widowed,?,Unmarried,Black,Female,0,4356,40,United-States,<=50K
3,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,<=50K
4,41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,22,Private,310152,Some-college,10,Never-married,Protective-serv,Not-in-family,White,Male,0,0,40,United-States,<=50K
32557,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32558,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32559,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K


In [27]:
# replacing '?' with NA values
print(income.isnull().sum())
print(income[income == '?'].count())

income.replace('?', pd.NA, inplace = True)

age               0
workclass         0
fnlwgt            0
education         0
education.num     0
marital.status    0
occupation        0
relationship      0
race              0
sex               0
capital.gain      0
capital.loss      0
hours.per.week    0
native.country    0
income            0
dtype: int64
age                  0
workclass         1836
fnlwgt               0
education            0
education.num        0
marital.status       0
occupation        1843
relationship         0
race                 0
sex                  0
capital.gain         0
capital.loss         0
hours.per.week       0
native.country     583
income               0
dtype: int64


In [28]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,13 mins 07 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.6
H2O_cluster_version_age:,1 day
H2O_cluster_name:,H2O_from_python_unknownUser_p2hamm
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.168 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


In [30]:
# splitting the dataset and converting DataFrames to H2OFrame
train_data, test_data = train_test_split(income, test_size = 0.2)

train_data_hf = h2o.H2OFrame(train_data)
test_data_hf = h2o.H2OFrame(test_data)

X = train_data_hf.columns
y = "income"
X.remove(y)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [31]:
# training the model
model = H2ORandomForestEstimator(ntrees = 1, sample_rate = 1, mtries = -2, max_depth = 10, min_rows = 1, balance_classes = True, min_split_improvement = 1e-5,  nfolds = 5)
model.train(x = X, y = y, training_frame = train_data_hf)

drf Model Build progress: |██████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ORandomForestEstimator : Distributed Random Forest
Model Key: DRF_model_python_1730666668118_18


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    1                  1                           4469                   10           10           10            333           333           333

ModelMetricsBinomial: drf
** Reported on train data. **

MSE: NaN
RMSE: NaN
LogLoss: NaN
Mean Per-Class Error: NaN
AUC: NaN
AUCPR: NaN
Gini: NaN

ModelMetricsBinomial: drf
** Reported on cross-validation data. **

MSE: 0.10573405797806608
RMSE: 0.3251677382183941
LogLoss: 0.767653745010149
Mean Per-Class Error: 0.1852684025589746
AUC: 0.8832813248652741
AUCPR: 0.7453918240360744
Gini: 0.7665626497305482

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.3545849215308738
       <=50K    >50K    Error    Rate
-----  -------  ------  -------  ----------------
<=50K  16480    3274    0.1657   (3274.0/19754.0)
>50K   1289     5005    0.2048   (1289.0/6294.0)
Total  17769    8279    0.1752   (4563.0/26048.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.354585     0.686887  143
max f2                       0.150403     0.774433  237
max f0point5                 0.628514     0.714029  64
max accuracy                 0.535336     0.85139   83
max precision                0.905393     0.925587  10
max recall                   0            1         399
max specificity              1            0.993571  0
max absolute_mcc             0.432213     0.581604  113
max min_per_class_accuracy   0.32391      0.817151  155
max mean_per_class_accuracy  0.23663      0.819481  189
max tns                      1            19627     0
max fns                      1            4848      0
max fps                      0            19754     399
max tps                      0            6294      399
max tnr                      1            0.993571  0
max fnr                      1            0.770257  0
max fpr                      0            1         399
max tpr                      0            1         399

Gains/Lift Table: Avg response rate: 24.16 %, avg score: 24.11 %
group    cumulative_data_fraction    lower_threshold    lift       cumulative_lift    response_rate    score        cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  ---------  -----------------  ---------------  -----------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0603885                   1                  3.80441    3.80441            0.919263         1            0.919263                    1                   0.229743        0.229743                   280.441   280.441            0.223314
2        0.113176                    0.751701           3.13325    3.49137            0.757091         0.786794     0.843623                    0.900557            0.165396        0.395138                   213.325   249.137            0.371801
3        0.150031                    0.629774           2.77196    3.31465            0.669792         0.697773     0.800921                    0.850743            0.102161        0.497299                   177.196   231.465            0.457915
4        0.208845                    0.526441           2.13681    2.98295            0.516319         0.550518     0.720772                    0.766194            0.125675        0.622974       

In [32]:
# predicting on the train and test sets
y_train_pred = model.predict(train_data_hf)
y_train_pred = y_train_pred.as_data_frame()
y_train = train_data['income'].values
y_train_pred = y_train_pred['predict'].values

y_test_pred = model.predict(test_data_hf)
y_test_pred = y_test_pred.as_data_frame()
y_test = test_data['income'].values
y_test_pred = y_test_pred['predict'].values

drf prediction progress: |███████████████████████████████████████████████████████| (done) 100%


/usr/local/lib/python3.10/dist-packages/h2o/frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


drf prediction progress: |███████████████████████████████████████████████████████| (done) 100%


/usr/local/lib/python3.10/dist-packages/h2o/job.py:81: UserWarning: Test/Validation dataset column 'native.country' has levels not trained on: ["Holand-Netherlands"]
  warnings.warn(w)
/usr/local/lib/python3.10/dist-packages/h2o/frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


In [33]:
# model evaluatation
f1_train = f1_score(y_train, y_train_pred, average = 'weighted')
f1_test = f1_score(y_test, y_test_pred, average = 'weighted')

print("f1 Score on Training Set:", f1_train)
print("f1 Score on Test Set:", f1_test)

f1 Score on Training Set: 0.8670005971676996
f1 Score on Test Set: 0.8561516789853941
